# Readme: for fixed tau=1000; PY/init t_life=1000;   <br /> sk1: 3p**h5 download ndraw=5 calculate locally (not 60 on cluster)

In [2]:
# f_seed 需改 PY/init 中的 因为 models_logd 传入的是 global 的f_seed 值
from PYmodule import *
from PYmodule.models_3p import *

from matplotlib.legend_handler import HandlerLine2D, HandlerTuple

In [3]:
print(f_seed)
! ls -lth ../3p/*h5

0.01
-rw-rw-r--@ 1 wli  staff    10M Nov 26 15:15 ../3p/M0fUt1000_f2_n1000.h5
-rw-rw-r--  1 wli  staff    49M Mar 15  2022 ../3p/MLF3prange1_l0_9.0e-01_a_1.0e-01.h5
-rw-rw-r--  1 wli  staff    48M Mar  7  2022 ../3p/M30err.5LF_3prange8_r_4even_ns5.0e+03.h5
-rw-rw-r--  1 wli  staff   144M Mar  6  2022 ../3p/M30LF_3prange8_r_4even_ns5.0e+03.h5


In [4]:
# best-fit paras and chi2s
fname = '../3p/M0fUt1000_f2_n1000.h5'
reader = emcee.backends.HDFBackend(fname)

Nburnin = 0
Nthin = 1
samples = reader.get_chain(discard=Nburnin)
probs = reader.get_log_prob(discard=Nburnin)
print('len of samples:', len(samples))

samples = reader.get_chain(discard=Nburnin, thin=Nthin, flat=True)
probs = reader.get_log_prob(discard=Nburnin, thin=Nthin, flat=True)
print('len of extracted samples:', len(samples))
theta_max = samples[np.argmax(probs)]
print('best paras:',theta_max,'chi2={:.2f}'.format(np.max(probs)*-2))


len of samples: 1000
len of extracted samples: 100000
best paras: [-0.52534832  0.12259319  0.38407638] chi2=32.81


In [5]:
f_seedlabel = 'f%d'%abs(int(np.log10(f_seed)))
f_seedlegend = r'$\mathrm{f_{seed}}=$'+str(f_seed)
labels = [r'$\tau$', r'$\log \delta$', r'$\lambda_0$', r'$\alpha$']
ndim = len(labels)


## corner

## generate MF & LF data  (moved to h5z6MFLF.py) ndraw=60, on cluster

In [5]:
# 
ndraw = 5

best_model = model(theta_max)

draw = np.floor(np.random.uniform(0,len(samples),size=ndraw)).astype(int)
thetas = samples[draw]
model_thetas = [model(theta_i) for theta_i in thetas]

In [6]:
y_best


NameError: name 'y_best' is not defined

In [7]:
curve_name = 'MF'
xs = best_model['M_BH'][::int(N_mf/100)] # ~100 points
y_data = best_model[curve_name+'_data'][::int(N_mf/100)]
y_logdata = np.log10(y_data)
y_best = best_model[curve_name][::int(N_mf/100)]
y_err = best_model[curve_name+'_data_err'][::int(N_mf/100)]

mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name][::int(N_mf/100)]
    mod_list.append(mod)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)

fMname = 't1000'+f_seedlabel+'ndraw%d'%ndraw+curve_name
ascii.write(Table([xs,y_data,y_best,y_err,med_model,spread]),
            fMname,
            names=['xs','y_data','y_best','y_err','med_model','spread'],
            formats={'xs':'10.3e','y_data':'10.3e','y_best':'10.3e','y_err':'10.3e','med_model':'10.3e','spread':'10.3e'},
            overwrite=True)

In [8]:
curve_name = 'LF'
xs = best_model['M1450']
Nappend = len(best_model['M1450'])-len(best_model['M1450_data'])
x_data = np.append(best_model['M1450_data'],np.ones(Nappend))
y_data = np.append(best_model[curve_name+'_data'],np.ones(Nappend))
y_data_err = np.append(best_model[curve_name+'_data_err'],np.ones(Nappend))

y_best = best_model[curve_name]
# spread
mod_list = []
for i in range(ndraw):
    mod = model_thetas[i][curve_name]
    mod_list.append(mod)
spread = np.std(mod_list,axis=0)
med_model = np.median(mod_list,axis=0)

fLname = 't1000'+f_seedlabel+'ndraw%d'%ndraw+curve_name
ascii.write(Table([xs,x_data,y_data,y_data_err,y_best,med_model,spread]),
            fLname,
            names=['xs','x_data','y_data','y_data_err','y_best','med_model','spread'],
            formats={'xs':'10.3e','x_data':'10.3e','y_data':'10.3e','y_data_err':'10.3e','y_best':'10.3e','med_model':'10.3e','spread':'10.3e'},
            overwrite=True)

In [ ]:
# 只为画图可先来这里直接read file 然后跑

In [6]:
ndraw = 5
fslabel,fslegend

(23, 20)

In [8]:
curve_name = 'MF'
fMname = 't1000'+f_seedlabel+'ndraw%d'%ndraw+curve_name
T_ = ascii.read(fMname, guess=False, delimiter=' ')
xs = T_['xs']
y_data = T_['y_data']
y_logdata = np.log10(y_data)
y_best = T_['y_best']
y_err = T_['y_err']
med_model = T_['med_model']
spread    = T_['spread']

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(xs, y_data, c='C0',label='W10')
# error band of W10
ax.fill_between(xs,pow(10.,y_logdata-y_err),pow(10.,y_logdata+y_err),color='C0',alpha=0.2,label='_data error')


ax.fill_between(xs,med_model-spread,med_model+spread,color='grey',alpha=0.5,label=r'_$1\sigma$ Posterior Spread')
ax.plot(xs, y_best, c='black', label='Best-fit Model')

ax.text(1e9,1e-7, f_seedlegend+'\n'+ \
labels[0]+' = %d Myr\n\n'%(1000)+labels[1]+' = %.2f\n'%(theta_max[0])\
+labels[2]+' = %.2f\n'%(theta_max[1])+labels[3]+' = %.2f\n'%(theta_max[2])
, fontsize=21)

ax.set_xlim(1e7,1e10); ax.set_xscale('log')
ax.set_ylim(1e-10,1e-4); ax.set_yscale('log')
ax.legend(fontsize=fslegend,loc='upper left')
plt.xlabel(r'$\mathrm{M_{\bullet}~(M_\odot)}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi_{M_{\bullet}}~(Mpc^{-3}dex^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)

plt.savefig('t1000'+f_seedlabel+'ndraw%dMF_spread.pdf'%ndraw,dpi=300,bbox_inches='tight')


# ---------------------------------         QLF         ----------------------------------------


In [13]:
# G15 3 data points 
M1450_G = np.array([-19,-20,-21]); logPhi_G = np.array([-5.2,-5.1,-5.7])
logPhicorr_G = np.array([-4.7,-4.7,-5.7])
print( M1450_G, pow(10,logPhi_G) )

# Giallongo 2019; X-ray faint QLF 4 data points
M1450_G19 = np.array([-19,-20,-21,-22]); Phi_G19 = np.array([7.27,4.77,0.69,0.62])*1e-6
Phip_G19 = np.array([7.12,3.79,1.61,1.44])*1e-6; Phim_G19 = np.array([4.02,2.31,0.60,0.54])*1e-6
# QLF sample: z=5.5 to 6.1; z=5.55 to z=6; extrapolating w/ Jiang2016 density evolution slope: -0.72
Phi_G19 *= pow(10, -0.72*(6-5.55))
Phip_G19*= pow(10, -0.72*(6-5.55))
Phim_G19*= pow(10, -0.72*(6-5.55))

# Jiang 2022;
namelist = [#'fig3a_data_individualFields',
           'fig3a_data_combinedFields_95CL',
           #'fig3a_data_combinedFields_75CL',
           ]
nameFitlist = [#'fig3a_data_modelFit_75CL',
           'fig3a_data_modelFit_95CL',
           ]
name = 'fig3a_data_combinedFields_95CL'
T_J = ascii.read('../data/Jiang_SD_Fig3/%s.txt'%(namelist[0]), guess=False, delimiter=' ',names=['M1450','Phi_cumu'])
M1450_J = (T_J['M1450'][1:] + T_J['M1450'][:-1]) /2.
dM1450_J = T_J['M1450'][1:] - T_J['M1450'][:-1]
Phi_J = 1e-9* (T_J['Phi_cumu'][1:] - T_J['Phi_cumu'][:-1])/dM1450_J

print(M1450_J,'\n', dM1450_J)

[-19 -20 -21] [6.30957344e-06 7.94328235e-06 1.99526231e-06]
       M1450       
-------------------
            -21.125
-20.435000000000002
            -19.705 
       M1450       
------------------
0.7100000000000009
0.6699999999999982
0.7900000000000027


In [15]:
curve_name = 'LF'
fLname = 't1000'+f_seedlabel+'ndraw%d'%ndraw+curve_name
T_ = ascii.read(fLname, guess=False, delimiter=' ')

xs = T_['xs']
x_data = T_['x_data']
y_data = T_['y_data']
y_data_err = T_['y_data_err']
y_best = T_['y_best']
med_model = T_['med_model']
spread    = T_['spread']


fig, ax = plt.subplots(figsize=(10, 8))
# Matsuoka 2018; data(errorbar) + fitting curve
p1=ax.errorbar(x_data, y_data/1e9, yerr=y_data_err/1e9,fmt='o',capsize=10, label='M18')
p1_,=ax.plot(xs,LF_M1450(xs),':',lw=2.5,c='C0',label='_Matsu fitting')

ax.fill_between(xs,(med_model-spread)/1e9,(med_model+spread)/1e9,color='grey',alpha=0.5,label='_',zorder=5)
p2,=ax.plot(xs, y_best/1e9, c='black', label='unobscured')
p3,=ax.plot(xs, y_best/1e9*corr_U14D20(xs),'--', c='black', label='intrinsic',zorder=5)


# Giallongo 2015; data + Poission errors
# ax.errorbar(M1450_G, pow(10,logPhicorr_G), yerr=pow(10,logPhicorr_G)/np.array([1,3**.5,1]),fmt='s',capsize=10, label='Giallongo+15')
# Giallongo 2019; Phi_corr data + errors
ax.errorbar(M1450_G19, Phi_G19, yerr=[Phim_G19,Phip_G19],fmt='s',capsize=10, label='_Giallongo+19')

# Jiang 2022; upper limits
# for name in namelist:
#     T_J = ascii.read('../data/Jiang_SD_Fig3/%s.txt'%name, guess=False, delimiter=' ',names=['M1450','Phi'])
#     M1450_J = T_J['M1450']
#     logPhi_J = T_J['Phi']/1e9
ax.errorbar( M1450_J, Phi_J,xerr=dM1450_J/2, yerr = Phi_J/2,uplims=True, label='_Jiang+ 22',fmt='D',color='C2',capsize=3)

# for name in nameFitlist:
#     T_J = ascii.read('../data/Jiang_SD_Fig3/%s.txt'%name, guess=False, delimiter=' ',names=['M1450','Phi'])
#     M1450_J = T_J['M1450']
#     logPhi_J = T_J['Phi']/1e9
#     ax.plot( M1450_J, logPhi_J, c='C2')

ax.text(-26,5e-8, f_seedlegend, fontsize=21)

ax.set_xlim(-18.5,-30)
ax.set_ylim(1e-12,1e-5)
ax.set_yscale('log')
plt.xlabel(r'$\mathrm{M_{1450}}$',fontsize=fslabel)
plt.ylabel(r'$\mathrm{\Phi_{M_{1450}}~(Mpc^{-3}mag^{-1})}$',fontsize=fslabel)
plt.tick_params(labelsize=fstick)
# plt.tight_layout()
plt.legend([p1,p2,p3],
           ['M18','unobscured','intrinsic'],
           fontsize=fslegend)

first_legend = ax.legend([(p1,p1_),p2,p3],
                         ['M18','unobscured','intrinsic'],
#                            numpoints=1,
#                            handler_map={tuple: HandlerTuple(ndivide=1,pad=.5,xpad=0,ypad=.7)},
                         handler_map={tuple: HandlerTuple(ndivide=1,xpad=0.1,ypad=-.3)},
                         fontsize=fslegend,loc=(.6,.73))
ax.add_artist(first_legend)

locmajx = FixedLocator(np.arange(-17,-30,-2)) # subs=(0.2,0.4,0.6,0.8)
locminx = FixedLocator(np.arange(17,-30,-.5)) # subs=(0.2,0.4,0.6,0.8)
ax.xaxis.set_major_locator(locmajx)
ax.xaxis.set_minor_locator(locminx)
ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

locmajy = LogLocator(base=10,numticks=100)
locminy = LogLocator(base=10,subs=np.arange(2, 10) * .1,numticks=100) # subs=(0.2,0.4,0.6,0.8)
ax.yaxis.set_major_locator(locmajy)
ax.yaxis.set_minor_locator(locminy)
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

plt.savefig('t1000'+f_seedlabel+'ndraw%dLF_spread.pdf'%ndraw,dpi=300,bbox_inches='tight')

In [13]:
print(datapre)

../data/


In [14]:
! ls datapre + 'M0'

ls: +: No such file or directory
ls: M0: No such file or directory
ls: datapre: No such file or directory


# prev